In [1]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

### 基于位置的前馈网络

In [2]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈神经网络"""
    
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        # super(FooChild,self) 首先找到 FooChild 的父类（就是类 FooParent），然后把类 FooChild 的对象转换为类 FooParent 的对象
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
        
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))
    

In [3]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
print(ffn(torch.ones((2, 3, 4)))[0])
print(ffn(torch.ones((2, 3, 4)))[1])


tensor([[-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439],
        [-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439],
        [-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439]],
       grad_fn=<SelectBackward0>)
tensor([[-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439],
        [-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439],
        [-0.1660,  0.0353, -0.4384, -0.0043,  0.1862, -0.1378, -0.2869, -0.2439]],
       grad_fn=<SelectBackward0>)


### 残差连接和层规范化

In [5]:
ln = nn.LayerNorm(2)
bn =  nn.BatchNorm1d(2)
X = torch.tensor([[1, 2],
                  [2, 3]], dtype=torch.float32)
# 在训练模式下计算 X 的均值和方差
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


In [6]:
class AddNorm(nn.Module):
    """残差链接后进行层规范化"""
    
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
        
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [7]:
# for a test
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

### 编码器

In [11]:
class EncoderBlock(nn.Module):
    """Transformer 编码块"""
    
    def __init__(self, query_size, key_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias=False, **kwargs): # 这里我把 query_size  放前面了，我觉得这样顺眼一些
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(query_size, key_size, value_size, num_hiddens, num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens) 
        self.addnorm2 = AddNorm(norm_shape, dropout)
        
    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))
        

In [12]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

torch.Size([2, 100, 24])

In [15]:
#@save
class TransformerEncoder(d2l.Encoder):
    """Transformer编码器"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，(NOTE:这是为什么呢？)
        # 然后再与位置编码相加。
        print(self.embedding(X).shape)
        print(self.embedding(X))
        print(self.embedding(X) * math.sqrt(self.num_hiddens))
        
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[
                i] = blk.attention.attention.attention_weights
        return X

In [16]:
encoder = TransformerEncoder(
    200, 24, 24, 24, 24, [100, 24], 24, 48, 8, 2, 0.5)
encoder.eval()
encoder(torch.ones((2, 100), dtype=torch.long), valid_lens).shape

torch.Size([2, 100, 24])
tensor([[[ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         ...,
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176]],

        [[ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         ...,
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176],
         [ 1.5149,  1.3712, -0.8276,  ..., -0.4783, -0.7301,  1.0176]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 7.4213,  6.7176, -4.0542,  ..., -2.3432, -3

torch.Size([2, 100, 24])